In [1]:
from time import strftime, strptime, localtime
from datetime import datetime as dt

In [ ]:
date1 = '2016-08-13'
date2 = '2017-08-10'

In [ ]:
strftime('%d-%m-%Y', strptime(date1, '%Y-%m-%d'))

In [ ]:
strptime(date1, '%Y-%m-%d') > strptime(date2, '%Y-%m-%d')

In [ ]:
strptime(date1, '%Y-%m-%d') < strptime(date2, '%Y-%m-%d')

In [ ]:
date1 + date2

In [ ]:
date1 < date2

In [ ]:
dt(2018, 12, 23)

In [ ]:
dt.fromtimestamp(int(dt.timestamp(dt(2018, 12, 22, 23, 59, 59)))).strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
strptime(strftime('%Y-%m-%d', localtime(1524686549464/1e3)), '%Y-%m-%d')

In [ ]:
int(dt.timestamp(dt(2018, 12, 22, 23, 59, 59)) * 1e3) 

# Reading DFs

In [3]:
import pandas as pd

In [4]:
from config.constants import LEAGUES_DATA_DICT, EXCEL_EXPORT_PATH_MERGED, EXCEL_EXPORT_PATH, SQL_LEAGUES_ENGINE
from sqlalchemy import create_engine

In [5]:
df = pd.read_excel(LEAGUES_DATA_DICT['SOLOQ'][EXCEL_EXPORT_PATH_MERGED])

In [14]:
df1 = pd.read_excel(LEAGUES_DATA_DICT['SOLOQ'][EXCEL_EXPORT_PATH])

In [10]:
len(df.gameId.unique()), df.shape[0]

(17, 171)

In [17]:
df.groupby('gameId').first().gameVersion.value_counts()

8.1.214.5847    12
8.1.213.4336     5
Name: gameVersion, dtype: int64